In [1]:
#Library


from pandasql import sqldf
import pandas as pd
import numpy as np
import openpyxl
import datetime
from openpyxl import workbook,load_workbook
from sqlalchemy import null
from zmq import NULL
mysql = lambda q: sqldf(q,globals()) 
from datetime import datetime as dt


In [2]:
#Importing booking dump as df1 and TA Portfolio as df2 
df1 = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\BOOKING DUMP\Booking-01082023.xlsx") #BookingDump Data 
df1['PromoCode']=df1['PromoCode'].str.upper()
df1['PromoCode']=df1['PromoCode'].str.strip()

df2 = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\TA PORTFOLIO\TAPortfolio-30042023.xlsx") #TA Portfolio data

df2['Region']=df2['Region'].str.strip()
df2['City']=df2['City'].str.strip()
df2['Agent Id - IATA Id']=df2['Agent Id - IATA Id'].str.strip()
df2['Parent Id']=df2['Parent Id'].str.strip()
df2['NEW Main Channel for Commercial Team']=df2['NEW Main Channel for Commercial Team'].str.strip()

df2 = df2.drop_duplicates(keep='first')

AttributeError: Can only use .str accessor with string values!

In [3]:
#Importing Corporate Portfolio as df3
df3 = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\CORPORATE PORTFOLIO\CorporatePortfolio-30042023.xlsx")

df3.rename(columns={'Promo code':'PromoCode'},inplace=True)
df3['PromoCode']=df3['PromoCode'].str.upper() 
df3['PromoCode']=df3['PromoCode'].str.strip()

df3 = df3.drop_duplicates(keep='first')

In [4]:
#Chartered Flight Name
Chatterflight=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChatterFlightMaster.xlsx")

#DOM/INT Master 
domint=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx",sheet_name='INT')

In [5]:
#Removing this SOC because it's a test id. Needs to be removed
booking = sqldf("select * from df1 where SourceOrganizationCode not in ('DELS0000','DELI0000','GOMARINE')")

#Copying Values to SourceOrganizationCode where PaymentOrganizationCode is not null
poc = sqldf("""Select *,
             CASE WHEN
             SourceOrganizationCode = 'G8' and PaymentOrganizationCode not NULL
             THEN 
             PaymentOrganizationCode
             WHEN
             SourceOrganizationCode = 'INFINITY' and PaymentOrganizationCode not NULL
             THEN 
             PaymentOrganizationCode
             ELSE
             SourceOrganizationCode END AS SOC 
             from booking """)


In [6]:
#creating some new variables
poc['Pax'] =  np.where(booking.PassengerStatus != "Cancelled",1,0)
poc['GrossRevenue'] = poc['BaseFare']+poc['ServiceChargeAmount']+poc['OtherTaxes']+poc['FuelTax']
poc['Netrevenue'] = poc['BaseFare'] + poc['FuelTax']
poc['GrossRevenuecrs'] = (poc['GrossRevenue'])/(10**7)
poc['NetRevenuecrs'] = (poc['Netrevenue'])/(10**7)
poc['sector'] = poc["DepartureStation"] + "-" + poc["ArrivalStation"]
poc['sector1'] = poc['FlightNumber'] + "-" + poc["DepartureStation"] + "-" + poc["ArrivalStation"]
poc['DepartureDate'] = pd.to_datetime(poc['DepartureDate']).dt.strftime('%d-%m-%Y')
poc['BookingDate'] = pd.to_datetime(poc['BookingDate']).dt.strftime('%d-%m-%Y')
poc['ModifiedDate'] = pd.to_datetime(poc['ModifiedDate']).dt.strftime('%d-%m-%Y')
poc['CancelledDate'] = pd.to_datetime(poc['CancelledDate']).dt.strftime('%d-%m-%Y')
poc['BookingMonth']=pd.to_datetime(poc['BookingDate'],format="%d-%m-%Y").dt.strftime('%b-%y')
poc['DepartureMonth'] = pd.to_datetime(poc['DepartureDate'],format="%d-%m-%Y").dt.strftime('%b-%y')

poc['DateDifference'] = (pd.to_datetime(poc['DepartureDate'], format="%d-%m-%Y") - pd.to_datetime(poc['BookingDate'],format="%d-%m-%Y")) / np.timedelta64(1, 'D')

#copying values from SOC to 'Agent Id - IATA Id' (column name same as in TA portfolio)
poc['Agent Id - IATA Id'] = poc['SOC'].copy()


In [7]:
#categorizing the flights as domestic or international flights
poc1 = sqldf("""select * , case when (DepartureStation in (select AirportCode from domint))  then 'INT'
                              when (ArrivalStation in (select AirportCode from domint)) then 'INT'
                   else 'DOM' end as domint from poc""")


In [8]:
# Applying left join on Bookingdump with TA portfolio
res1 = pd.merge(poc1,df2,on='Agent Id - IATA Id',how = 'left')

# renaming the column eradicating space and adding further calculation columns on booking dump
res1.rename(columns={"Agent Id - IATA Id" : "AgentId","Parent Id":"ParentId","ID Type":"IDType","TA Name":"TAName",
                     "Main Channel":"MainChannel","Main Channel on the basis of Parent ID":"MainChannelBasedOnParentID",
                     "Country_Name":"CountryName","Global_Region":"GlobalRegion",
                     "NEW Main Channel for Commercial Team":"NEWMainChannel" },inplace=True)


In [9]:
#Need to change this departure month every first of the new month
res1['DepMon']=pd.to_datetime(res1['DepartureMonth'],format="%b-%y").dt.strftime("%m-%y")

#Manually Update the date here every month
res1['MTD/FWD'] = np.where(res1['DepMon'] != "08-23","FWD","MTD")
res1['FIT/Grp_Series'] = np.where(res1['FareClass'] != "G","FIT","Grp & Series")


In [10]:
#Replacing the values of NEWMainChannel with ChannelType where NEWMainChannel is NULL
res2 = sqldf(""" Select *,
              CASE when NEWMainChannel is NULL
              THEN ChannelType
              ELSE NEWMainChannel
              END as NEWMainChannel1
              from res1
              """)
res2['NEWMainChannel1'] = res2['NEWMainChannel1'].str.upper()


In [11]:
Chatterflight['combokey']=Chatterflight['FltNo'].astype(str)+Chatterflight['Org']+Chatterflight['Dest']

In [12]:
#Separating the chartered flights in a new dataframe
split=res2["FlightNumber"].str.split("/", n = -1, expand = True)

if(len(split.columns)==1):
    res2['Flight1'] = split[0]
    res2['combokey1'] = res2['Flight1']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight2'] = ''
    res2['combokey2'] = ''
    res2['Flight3'] = ''
    res2['combokey3'] = ''
    res2['Flight4'] = ''
    res2['combokey4'] = ''
elif(len(split.columns)==2):
    res2['Flight1'] = split[0]
    res2['combokey1'] = res2['Flight1']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight2'] = split[1]
    res2['combokey2'] = res2['Flight2']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight3'] = ''
    res2['combokey3'] = ''
    res2['Flight4'] = ''
    res2['combokey4'] = ''
elif(len(split.columns)==3):
    res2['Flight1'] = split[0]
    res2['combokey1'] = res2['Flight1']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight2'] = split[1]
    res2['combokey2'] = res2['Flight2']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight3'] = split[2]
    res2['combokey3'] = res2['Flight3']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight4'] = ''
    res2['combokey4'] = ''
else:
    res2['Flight1'] = split[0]
    res2['combokey1'] = res2['Flight1']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight2'] = split[1]
    res2['combokey2'] = res2['Flight2']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight3'] = split[2]
    res2['combokey3'] = res2['Flight3']+res2['DepartureStation']+res2['ArrivalStation']
    res2['Flight4'] = split[3]
    res2['combokey4'] = res2['Flight4']+res2['DepartureStation']+res2['ArrivalStation']
    
res3 = sqldf("""select *,case when (combokey1 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey2 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey3 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey4 in (Select combokey from Chatterflight)) then 'Charter'
                                                  else 'Non-Charter' end as charter_noncharter from res2""")


In [13]:
#Final table in which AgentId is INFINITY and ChannelType is API is changed to DIRECT in NEWMainChannel2
result = sqldf(""" Select *,
                CASE when AgentId = 'INFINITY' and ChannelType = 'API' THEN 'DIRECT'
                ELSE NEWMainChannel1
                END as NEWMainChannel2
                from res3 
                """)


In [14]:
#Dataset where the Region is converted to ChannelType if it is NULL under the column name Region1
res2a = sqldf(""" Select *,
              CASE when Region is NULL
              THEN ChannelType
              ELSE Region
              END as Region1
              from result
              """)

res2a['Region1'] = res2a['Region1'].str.upper()


In [15]:
#Final table in which AgentId is INFINITY and ChannelType is API is changed to DIRECT in Region2
resulta = sqldf(""" Select *,
                CASE when AgentId = 'INFINITY' and ChannelType = 'API' THEN 'DIRECT'
                ELSE Region1
                END as Region2
                from res2a 
                """)


In [16]:
#Merging the Corporate portfolio with the current dataset
results = pd.merge(resulta,df3,on='PromoCode', how = 'left')
results.rename(columns={'CITY':'CITY_1'},inplace=True)


In [17]:
#creating a new variable called CurveBand based on the Date Difference values
resultfinal = sqldf("""select *, case when DateDifference>60 then "D60+" 
                                      when  DateDifference<=60 and DateDifference>45 then "D46-D60"
                                      when DateDifference<=45 and DateDifference>30 then "D31-D45"
                                      when DateDifference<=30 and DateDifference>15 then "D16-D30"
                                      when DateDifference<=15 and  DateDifference>7 then "D8-D15" 
                                      when DateDifference<=7 and DateDifference>3 then "D04-D07" 
                                      else "D00-D03" end as CurveBand from results """)


In [18]:
resultfinal['AgentRole'] = ''
#resultfinal['Remarks'] = ''
resultfinal['BkgDate']=resultfinal['BookingDate'].copy()
resultfinal['BkgMonth']=resultfinal['BookingMonth'].copy()
resultfinal['DepMonth']=resultfinal['DepartureMonth'].copy()

#resultfinal.columns

#changing all the API values in the data to DIRECT and converting the columns to uppercase
resultfinal['NEWMainChannel2']=sqldf("""select case when NEWMainChannel2 = 'API' then 'DIRECT' else NEWMainChannel2 end as NEWMainChannel2 from resultfinal""")
resultfinal['NEWMainChannel2']=resultfinal['NEWMainChannel2'].str.upper()

resultfinal['ChannelType']=resultfinal['ChannelType'].str.upper()

resultfinal['CORPORATE']=sqldf("""select case when [PC/NPC] = 'NPC' then 'NA' else CORPORATE end as CORPORATE from resultfinal""")
resultfinal['NEW BDM']=sqldf("""select case when [PC/NPC] = 'NPC' then 'NA' else [NEW BDM] end as [NEW BDM] from resultfinal""")
resultfinal['CM_y']=sqldf("""select case when [PC/NPC] = 'NPC' then 'NA' else CM_y end as CM_y from resultfinal""")
resultfinal['CITY_1']=sqldf("""select case when [PC/NPC] = 'NPC' then 'NA' else CITY_1 end as CITY_1 from resultfinal""")
resultfinal['Region_y']=sqldf("""select case when [PC/NPC] = 'NPC' then 'NA' else Region_y end as Region_y from resultfinal""")

resultfinal['ParentId']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else ParentId end as ParentId from resultfinal""")
resultfinal['ParentId']=resultfinal['ParentId'].str.upper()

resultfinal['IDType']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else IDType end as IDType from resultfinal""")
resultfinal['IDType']=resultfinal['IDType'].str.upper()

resultfinal['City']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else City end as City from resultfinal""")
resultfinal['City']=resultfinal['City'].str.upper()

resultfinal['TAName']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else TAName end as TAName from resultfinal""")
resultfinal['TAName']=resultfinal['TAName'].str.upper()

resultfinal['Channel']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else Channel end as Channel from resultfinal""")
resultfinal['Channel']=resultfinal['Channel'].str.upper()

resultfinal['MainChannel']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else MainChannel end as MainChannel from resultfinal""")
resultfinal['MainChannel']=resultfinal['MainChannel'].str.upper()

resultfinal['Country_Name ']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else [Country_Name ] end as [Country_Name ] from resultfinal""")
resultfinal['Country_Name ']=resultfinal['Country_Name '].str.upper()

resultfinal['GlobalRegion']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else GlobalRegion end as GlobalRegion from resultfinal""")
resultfinal['GlobalRegion']=resultfinal['GlobalRegion'].str.upper()

resultfinal['Main Channel \non the basis of\n Parent ID']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType 
                                                                else [Main Channel \non the basis of\n Parent ID] end as [Main Channel \non the basis of\n Parent ID] from resultfinal""")
resultfinal['Main Channel \non the basis of\n Parent ID']=resultfinal['Main Channel \non the basis of\n Parent ID'].str.upper()

resultfinal['NEWMainChannel2']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else NEWMainChannel2 end as NEWMainChannel2 from resultfinal""")
resultfinal['NEWMainChannel2']=resultfinal['NEWMainChannel2'].str.upper()

In [19]:
#converting the other NULL values to ChannelType in other columns as well
resultfinal.rename(columns={"Main Channel \non the basis of\n Parent ID":'MainChannelParentID'},inplace=True)

resultfinaldata=sqldf("""select *,
                            case when Region2 is NULL then ChannelType else Region2 end as Region2,
                            case when City is NULL then ChannelType else City end as City,
                            case when GlobalRegion is NULL then ChannelType else GlobalRegion end as GlobalRegion,
                            case when Channel is NULL then ChannelType else Channel end as Channel,
                            case when MainChannel is NULL then ChannelType else MainChannel end as MainChannel,
                            case when MainChannelParentID is NULL then ChannelType else MainChannelParentID end as MainChannelParentID,
                            case when [Country_Name ] is NULL then ChannelType else [Country_Name ] end as [Country_Name ],
                            case when TAName is NULL then NEWMainChannel2 else TAName end as TAName,
                            case when ParentId is NULL then ChannelType else ParentId end as ParentId,
                            case when IDType is NULL then ChannelType else IDType end as IDType,
                            case when NEWMainChannel2 is NULL then ChannelType else NEWMainChannel2 end as NEWMainChannel2
                            from resultfinal""")

#resultdata=sqldf("""select *, case when ChannelType='API' then 'DIRECT' else ChannelType end as ChannelType from resultfinaltest""")


In [20]:
#converting the API values if any to DIRECT
resultfinaltest=sqldf("""select *, 
                            case when Region2 = 'API' then 'DIRECT' else Region2 end as Region2,
                            case when City = 'API' then 'DIRECT' else City end as City, 
                            case when GlobalRegion = 'API' then 'DIRECT' else GlobalRegion end as GlobalRegion,
                            case when MainChannel = 'API' then 'DIRECT' else MainChannel end as MainChannel,
                            case when Channel = 'API' then 'DIRECT' else Channel end as Channel,
                            case when MainChannelParentID = 'API' then 'DIRECT' else MainChannelParentID end as MainChannelParentID,
                            case when ParentId = 'API' then 'DIRECT' else ParentId end as ParentId,
                            case when [Country_Name ] = 'API' then 'DIRECT' else [Country_Name ] end as [Country_Name ],
                            case when IDType = 'API' then 'DIRECT' else IDType end as IDType,
                            case when TAName = 'API' then 'DIRECT' else TAName end as TAName,
                            case when NEWMainChannel2 = 'API' then 'DIRECT' else NEWMainChannel2 end as NEWMainChannel2
                            from resultfinaldata""")

In [21]:
newdf = sqldf(""" select *,
                case when 
                [FIT/Grp_Series] = 'FIT' and IDType != 'GS'  then 'FIT'
                when 
                [FIT/Grp_Series] ='FIT' and IDType='GS'  then 'GSS'
                when 
                [FIT/Grp_Series] ='Grp & Series'  then 'GnS'
                end as Fit_GSS_Gns from resultfinaltest where charter_noncharter = 'Non-Charter'
                                        """)

In [22]:
#newdf.to_excel("fit_gss_gns.xlsx")

In [23]:
resultfin = sqldf(""" select 
                  FlightNumber, 
                  DepartureStation, 
                  ArrivalStation,                 
                  DepartureDate, 
                  BookingDate, 
                  ModifiedDate,
                  CancelledDate, 
                  ChannelType, 
                  FlightStatus,
                  PNR, 
                  PassengerLastNameAndFirstName, 
                  BaseFare,
                  ServiceChargeAmount, 
                  FuelTax, 
                  OtherTaxes,
                  FareClass, 
                  Cabin, 
                  FareBasis, 
                  PromoCode,
                  PromoCodeAmount, 
                  PassengerStatus, 
                  PassengerType,
                  BookingAgentID, 
                  SourceOrganizationCode,
                  SourceOrganizationCodeName,
                  PaymentOrganizationCode,
                  OrganizationGroup, 
                  PaymentOrganizationName,
                  productType,
                  ProductTypeName,
                  AgentRole, 
                  Pax, 
                  GrossRevenue  as   Gross_Rev,
                  Netrevenue  as   Net_Rev,
                  GrossRevenuecrs  as   Gross_Rev_inCrs, 
                  sector  as   Sector, 
                  sector1  as   Sector1, 
                  domint  as   DOM_INT,
                  BkgMonth  as   BkgMonth, 
                  DepMonth  as   DepMonth, 
                  BkgDate  as   BkgDate, 
                  AgentId  as   AgentID,
                  ParentId  as   ParentID, 
                  IDType, 
                  Region2  as   Region, 
                  City, 
                  TAName,
                  Channel as Sub_Channel, 
                  MainChannel as Sub_MainChannel, 
                  RGM, 
                  ASM, 
                  CM_x as CM,
                  BDM,
                  [PC/NPC]  as   PromoCodeType, 
                  CORPORATE  as   CorporateName, 
                  [NEW BDM]  as   BDM_Corp, 
                  CM_y  as   CM_Corp,
                  CITY_1  as   City_Corp,
                  Region_y  as   Region_Corp, 
                  [FIT/Grp_Series]  as   Grp_FIT,
                  NEWMainChannel2  as   MainChannel, 
                  [Country_Name ] as Country_Name,
                  GlobalRegion  as   Global_Region, 
                  DateDifference, 
                  CurveBand,
                  DepartureDate  as   DepDate, 
                  NetRevenuecrs  as   NetRevinCrs, 
                  MainChannelParentID as parentIDchannel, 
                  Flight1, Flight2, Flight3, Flight4, charter_noncharter, [MTD/FWD]
                  from resultfinaltest """)


In [24]:
#Generating the modified booking dumpcode
resultfin.to_excel(r"Booking_processed-01082023.xlsx",index=False)	


In [25]:
### New Lines of code
# CHMast=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx", sheet_name= 'Channel' )

# channelwise_fit = sqldf("""select MainChannel, sum(Pax) as FitPax, sum(Gross_Rev_inCrs) as FitRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as FITDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as FITDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as FITIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as FITIntRev
#           from resultfin where Grp_FIT='FIT' and IDType!='GS' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_fit_order = sqldf("""select a.ChnName as Channel, b.FitPax as [FIT DOM+INT Pax], b.FitRev as [FIT DOM+INT Rev],
#                                   b.FITDomPax as [FIT DOM Pax], b.FITDomRev as [FIT DOM Rev], b.FITIntPax as [FIT INT Pax], b.FITIntRev as [FIT INT Rev]
#                                   from CHMast a left join channelwise_fit b on a.ChnName = b.MainChannel""")
# # ##Exporting values to Excel
# with pd.ExcelWriter(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\Daily MIS Intermediary\IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
#      channelwise_fit_order.to_excel(writer, sheet_name="Fit_GSS_GS", startrow=1,startcol=1, index=False)


In [26]:
# GSS Pax and Revenue
# channelwise_gss = sqldf("""select MainChannel, sum(Pax) as GSSPax, sum(Gross_Rev_inCrs) as GSSRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as GSSDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as GSSDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as GSSIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as GSSIntRev
#           from resultfin where Grp_FIT='FIT' and IDType='GS' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_gss_order = sqldf("""select a.ChnName as Channel, b.GSSPax as [GSS DOM+INT Pax], b.GSSRev as [GSS DOM+INT Rev],
#                                   b.GSSDomPax as [GSS DOM Pax], b.GSSDomRev as [GSS DOM Rev], b.GSSIntPax as [GSS INT Pax], b.GSSIntRev as [GSS INT Rev]
#                                   from CHMast a left join channelwise_gss b on a.ChnName = b.MainChannel""")
# ##Exporting values to Excel
# with pd.ExcelWriter(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\Daily MIS Intermediary\IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
#      channelwise_gss_order.to_excel(writer, sheet_name="Fit_GSS_GS", startrow=11,startcol=1, index=False)


In [27]:
# GnS Pax and Revenue
# channelwise_gns = sqldf("""select MainChannel, sum(Pax) as GnSPax, sum(Gross_Rev_inCrs) as GnSRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as GnSDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as GnSDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as GnSIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as GnSIntRev
#           from resultfin where Grp_FIT='Grp & Series' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_gns_order = sqldf("""select a.ChnName as Channel, b.GnSPax as [GnS DOM+INT Pax], b.GnSRev as [GnS DOM+INT Rev],
#                                   b.GnSDomPax as [GnS DOM Pax], b.GnSDomRev as [GnS DOM Rev], b.GnSIntPax as [GnS INT Pax], b.GnSIntRev as [GnS INT Rev]
#                                   from CHMast a left join channelwise_gns b on a.ChnName = b.MainChannel""")
# ##Exporting values to Excel
# with pd.ExcelWriter(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\Daily MIS Intermediary\IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
#      channelwise_gns_order.to_excel(writer, sheet_name="Fit_GSS_GS", startrow=22,startcol=1, index=False)
 
